# Git Internals memo
## a personal memo for studying GIT PRO
#### https://git-scm.com/book/en/v2/Git-Internals-Git-Objects

In [1]:
import os, tempfile
os.chdir(tempfile.mkdtemp())
os.getcwd()

'/private/var/folders/nr/0t8pv_9x62d796lz8dhth24m0000gn/T/tmpf8ct5ykh'

In [2]:
!git init test
os.chdir('test')
os.getcwd()

Initialized empty Git repository in /private/var/folders/nr/0t8pv_9x62d796lz8dhth24m0000gn/T/tmpf8ct5ykh/test/.git/


'/private/var/folders/nr/0t8pv_9x62d796lz8dhth24m0000gn/T/tmpf8ct5ykh/test'

In [3]:
!find .git/objects

.git/objects
.git/objects/info
.git/objects/pack


In [4]:
hashobj = !echo 'test content' | git hash-object -w --stdin
hashobj

['d670460b4b4aece5915caf5c68d12f560a9fe3e4']

In [5]:
!find .git/objects -type f

.git/objects/d6/70460b4b4aece5915caf5c68d12f560a9fe3e4


In [6]:
!git cat-file -p {hashobj[0]}

test content


In [7]:
!echo 'version 1' > test.txt

In [8]:
ver1hash = !git hash-object -w test.txt
ver1hash

['83baae61804e65cc73a7201a7252750c76066a30']

In [9]:
!echo 'version 2' > test.txt
ver2hash = !git hash-object -w test.txt
ver2hash

['1f7a7a472abf3dd9643fd615f6da379c4acb3e3a']

In [10]:
!find .git/objects -type f

.git/objects/1f/7a7a472abf3dd9643fd615f6da379c4acb3e3a
.git/objects/83/baae61804e65cc73a7201a7252750c76066a30
.git/objects/d6/70460b4b4aece5915caf5c68d12f560a9fe3e4


In [11]:
!git cat-file -p {ver1hash[0]} > test.txt
!cat test.txt

version 1


In [12]:
! git cat-file -p {ver2hash[0]} > test.txt
! cat test.txt

version 2


In [13]:
! git cat-file -t {ver2hash[0]}

blob


In [14]:
#what does update-index do....
#to check more details from Pro GIT
#You can fairly easily create your own tree.
#Git normally creates a tree by taking the state of your
#staging area or index and writing a series of tree objects
#from it. So, to create a tree object, you first have 
#to set up an index by staging some files. To create an 
#index with a single entry – the first version of your 
#test.txt file – you can use the plumbing command
#update-index. You use this command to artificially add
#the earlier version of the test.txt file to a new 
#staging area. You must pass it the --add option 
#because the file doesn’t yet exist in your staging
#area (you don’t even have a staging area set up yet) 
#and --cacheinfo because the file you’re adding isn’t 
#in your directory but is in your database. Then, you
#specify the mode, SHA-1, and filename:

! git update-index --add --cacheinfo \
100644,{ver1hash[0]},test.txt

In [15]:
tree1 = !git write-tree
tree1

['d8329fc1cc938780ffdd9f94e0d364e0ea74f579']

In [16]:
! git cat-file -p {tree1[0]}

100644 blob 83baae61804e65cc73a7201a7252750c76066a30	test.txt


In [17]:
! git cat-file -t {tree1[0]}

tree


In [18]:
#You’ll now create a new tree with the second version of test.txt and a new file as well:
! echo 'new file' > new.txt
! git update-index test.txt
! git update-index --add new.txt

In [19]:
tree2 = !git write-tree
tree2

['0155eb4229851634a0f03eb265b69f5a2d56f341']

In [20]:
! git cat-file -p {tree2[0]}

100644 blob fa49b077972391ad58037050f2a75f74e3671e92	new.txt
100644 blob 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a	test.txt


In [21]:
! git read-tree --prefix=bak/ {tree1[0]}
tree3 = ! git write-tree
tree3

['3c4e9cd789d88d8d89c1073707c3585e41b0e614']

In [22]:
! git cat-file -p {tree3[0]}

040000 tree d8329fc1cc938780ffdd9f94e0d364e0ea74f579	bak
100644 blob fa49b077972391ad58037050f2a75f74e3671e92	new.txt
100644 blob 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a	test.txt


In [23]:
commit1 = !git commit-tree {tree1[0]} -m 'first commit'
commit1

['db751eb44b7329ddf1a011988e45e848ef7b79fb']

In [24]:
!git cat-file -p {commit1[0]}

tree d8329fc1cc938780ffdd9f94e0d364e0ea74f579
author ajinsusumu <ajinsusumu@gmail.com> 1478792257 +0800
committer ajinsusumu <ajinsusumu@gmail.com> 1478792257 +0800

first commit


In [25]:
commit2 = !git commit-tree {tree2[0]} -m 'second commit' -p {commit1[0]}
commit2

['bc383cff56f996a680ae8a8d21588f529a668e98']

In [26]:
commit3 = !git commit-tree {tree3[0]} -m 'third commit' -p {commit2[0]}
commit3

['74689953b9999b37f864dbc52131a4e1b6d9c499']

In [27]:
!git log --stat {commit3[0]}

commit 74689953b9999b37f864dbc52131a4e1b6d9c499
Author: ajinsusumu <ajinsusumu@gmail.com>
Date:   Thu Nov 10 23:37:38 2016 +0800

    third commit

 bak/test.txt | 1 +
 1 file changed, 1 insertion(+)

commit bc383cff56f996a680ae8a8d21588f529a668e98
Author: ajinsusumu <ajinsusumu@gmail.com>
Date:   Thu Nov 10 23:37:38 2016 +0800

    second commit

 new.txt  | 1 +
 test.txt | 2 +-
 2 files changed, 2 insertions(+), 1 deletion(-)

commit db751eb44b7329ddf1a011988e45e848ef7b79fb
Author: ajinsusumu <ajinsusumu@gmail.com>
Date:   Thu Nov 10 23:37:37 2016 +0800

    first commit

 test.txt | 1 +
 1 file changed, 1 insertion(+)


In [28]:
!find .git/objects -type f

.git/objects/01/55eb4229851634a0f03eb265b69f5a2d56f341
.git/objects/1f/7a7a472abf3dd9643fd615f6da379c4acb3e3a
.git/objects/3c/4e9cd789d88d8d89c1073707c3585e41b0e614
.git/objects/74/689953b9999b37f864dbc52131a4e1b6d9c499
.git/objects/83/baae61804e65cc73a7201a7252750c76066a30
.git/objects/bc/383cff56f996a680ae8a8d21588f529a668e98
.git/objects/d6/70460b4b4aece5915caf5c68d12f560a9fe3e4
.git/objects/d8/329fc1cc938780ffdd9f94e0d364e0ea74f579
.git/objects/db/751eb44b7329ddf1a011988e45e848ef7b79fb
.git/objects/fa/49b077972391ad58037050f2a75f74e3671e92


## Git References

In [29]:
!find .git/refs

.git/refs
.git/refs/heads
.git/refs/tags


In [30]:
!find .git/refs -type f

In [31]:
master=".git/refs/heads/master"
with open(master, "w") as outf: outf.write(commit3[0])

In [32]:
!git log --pretty=oneline master

74689953b9999b37f864dbc52131a4e1b6d9c499 third commit
bc383cff56f996a680ae8a8d21588f529a668e98 second commit
db751eb44b7329ddf1a011988e45e848ef7b79fb first commit


In [33]:
!git update-ref refs/heads/test {commit2[0]}

In [34]:
!git log --pretty=oneline test

bc383cff56f996a680ae8a8d21588f529a668e98 second commit
db751eb44b7329ddf1a011988e45e848ef7b79fb first commit


### The HEAD

In [35]:
!cat .git/HEAD

ref: refs/heads/master


In [36]:
!git checkout test

Switched to branch 'test'


In [37]:
!cat .git/HEAD

ref: refs/heads/test


In [38]:
!git checkout master

Switched to branch 'master'


In [39]:
!git symbolic-ref HEAD

refs/heads/master


In [40]:
!git symbolic-ref HEAD refs/heads/test
!cat .git/HEAD

ref: refs/heads/test


### Tags

In [41]:
!git update-ref refs/tags/v1.0 {commit2[0]}

In [42]:
!git tag -a v1.1 -m 'test tag' {commit3[0]}

In [43]:
!!cat .git/refs/tags/v1.1

['b9ac6e21d918b7b67170a741023fbcf9227334e4']

In [44]:
!git cat-file -p {_[0]}

object 74689953b9999b37f864dbc52131a4e1b6d9c499
type commit
tag v1.1
tagger ajinsusumu <ajinsusumu@gmail.com> 1478792259 +0800

test tag


In [45]:
!find .git/objects -type f

.git/objects/01/55eb4229851634a0f03eb265b69f5a2d56f341
.git/objects/1f/7a7a472abf3dd9643fd615f6da379c4acb3e3a
.git/objects/3c/4e9cd789d88d8d89c1073707c3585e41b0e614
.git/objects/74/689953b9999b37f864dbc52131a4e1b6d9c499
.git/objects/83/baae61804e65cc73a7201a7252750c76066a30
.git/objects/b9/ac6e21d918b7b67170a741023fbcf9227334e4
.git/objects/bc/383cff56f996a680ae8a8d21588f529a668e98
.git/objects/d6/70460b4b4aece5915caf5c68d12f560a9fe3e4
.git/objects/d8/329fc1cc938780ffdd9f94e0d364e0ea74f579
.git/objects/db/751eb44b7329ddf1a011988e45e848ef7b79fb
.git/objects/fa/49b077972391ad58037050f2a75f74e3671e92


In [46]:
!curl https://raw.githubusercontent.com/mojombo/grit/master/lib/grit/repo.rb > repo.rb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22044  100 22044    0     0  30981      0 --:--:-- --:--:-- --:--:-- 30960


In [47]:
!git checkout master

Switched to branch 'master'


In [48]:
!git add repo.rb

In [49]:
!git commit -m 'added repo.rb'

[master efc8f0d] added repo.rb
 1 file changed, 709 insertions(+)
 create mode 100644 repo.rb


In [50]:
!!git cat-file -p master^{tree} | grep repo.rb

['100644 blob 033b4468fa6b2a9547a70d88d1bbe8bf3f9ed0d5\trepo.rb']

In [51]:
!git cat-file -s {_[0].split()[2]}

22044


In [52]:
!echo '# testing' >> repo.rb
!git commit -am 'modified repo a bit'

[master 37dfa1c] modified repo a bit
 1 file changed, 1 insertion(+)


In [53]:
!!git cat-file -p master^{tree} | grep repo.rb

['100644 blob b042a60ef7dff760008df33cee372b945b6e884e\trepo.rb']

In [54]:
!git cat-file -s {_[0].split()[2]}

22054


In [55]:
!git gc

Counting objects: 16, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (12/12), done.
Writing objects: 100% (16/16), done.
Total 16 (delta 2), reused 0 (delta 0)


In [56]:
!git verify-pack -v .git/objects/pack/*.pack

37dfa1c7bac456bae09ca6a919c5f982de52ca93 commit 234 155 12
efc8f0dc6307b7331c95bc688c827bb69550e8fe commit 228 150 167
bc383cff56f996a680ae8a8d21588f529a668e98 commit 228 149 317
74689953b9999b37f864dbc52131a4e1b6d9c499 commit 227 147 466
b9ac6e21d918b7b67170a741023fbcf9227334e4 tag    137 123 613
db751eb44b7329ddf1a011988e45e848ef7b79fb commit 179 119 736
fe879577cb8cffcdf25441725141e310dd7d239b tree   136 136 855
d8329fc1cc938780ffdd9f94e0d364e0ea74f579 tree   36 46 991
deef2e1b793907545e50a2ea2ddb5ba6c58c4506 tree   136 136 1037
0155eb4229851634a0f03eb265b69f5a2d56f341 tree   71 76 1173
3c4e9cd789d88d8d89c1073707c3585e41b0e614 tree   8 19 1249 1 deef2e1b793907545e50a2ea2ddb5ba6c58c4506
83baae61804e65cc73a7201a7252750c76066a30 blob   10 19 1268
fa49b077972391ad58037050f2a75f74e3671e92 blob   9 18 1287
b042a60ef7dff760008df33cee372b945b6e884e blob   22054 5799 1305
033b4468fa6b2a9547a70d88d1bbe8bf3f9ed0d5 blob   9 20 7104 1 b042a60ef7dff760008df33cee372b945b6e884e
1f7a7a472abf3dd9643f

In [57]:
#os.chdir('..')

In [58]:
#os.environ['http_proxy']='http://127.0.0.1:3128'
#os.environ['https_proxy']='https://127.0.0.1:3128'

In [59]:
#!git clone https://github.com/schacon/simplegit.git

In [60]:
#os.chdir('simplegit')

In [61]:
#!git cat-file -p master^{tree}

In [62]:
#!git cat-file -p 99f1a6d12cb4b6f19c8655fca46c3ecf317074e0